In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
import os
import sys
import glob
import random

!pip install ipywidgets
from pprint import pprint
from ipywidgets import Video

from PIL import Image
from PIL.ExifTags import TAGS

   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 914.9/914.9 kB 35.2 MB/s  0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 59.0 MB/s  0:00:00

   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   ---------------------------------------- 3/3 [ipywidgets]



In [3]:
def create_tracker(tracker_type):
    tracker = None
    tracker_types = [
        "BOOSTING",
        "MIL",
        "KCF",
        "TLD",
        "MEDIANFLOW",
        "MOSSE",
        "CSRT",
    ]
    if tracker_type == 'BOOSTING':
        tracker = cv2.legacy.TrackerBoosting_create()
    if tracker_type == 'MIL':
        tracker = cv2.TrackerMIL_create()
    if tracker_type == 'KCF':
        tracker = cv2.TrackerKCF_create()
    if tracker_type == 'TLD':
        tracker = cv2.legacy.TrackerTLD_create()
    if tracker_type == 'MEDIANFLOW':
        tracker = cv2.legacy.TrackerMedianFlow_create()
    if tracker_type == 'MOSSE':
        tracker = cv2.legacy.TrackerMOSSE_create()
    if tracker_type == "CSRT":
        tracker = cv2.TrackerCSRT_create()
    return tracker

def draw_bbox(frame, bbox, color=(255, 255, 255)):
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, color, 2, 1)

# Object Identification & Tracking Pipeline (ORB + MOG2)

 - Builds a feature dataset (keypoints/descriptors) for board game objects
   from images in `media/photos/*`.
 - Uses background subtraction (MOG2) to find candidate regions in frames.
 - Computes ORB descriptors for candidates, matches to dataset, and if a
   confident match is found, initializes a CSRT tracker for that object.
 - Tracks all initialized objects across the video and writes an output.


In [ ]:
# Parameters & Setup
import os
import glob
import cv2
import numpy as np

# Directory with object images for the dataset
DATASET_DIR = os.path.join('.', 'media', 'photos')  # adjust if needed

# Path to the game video to process
VIDEO_PATH = os.path.join('.', 'media', 'game.mp4')  # replace with your video
OUTPUT_PATH = os.path.join('.', 'media', 'game_track.avi')

# Background subtractor (MOG2)
backSub = cv2.createBackgroundSubtractorMOG2(history=200, varThreshold=50, detectShadows=True)

# Morphology kernels
KERNEL_OPEN = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
KERNEL_CLOSE = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

# Minimum area to consider a contour a candidate (tune as needed)
MIN_AREA = 500

# ORB feature extractor
ORB = cv2.ORB_create(nfeatures=1000)

# BFMatcher for ORB (Hamming distance)
BF = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

In [ ]:
# Build feature dataset from images
def load_gray(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        return None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img, gray

def build_object_dataset(dataset_dir):
    dataset = {}  # name -> { 'image': img, 'gray': gray, 'kps': kps, 'des': des }
    image_paths = sorted(glob.glob(os.path.join(dataset_dir, '*.*')))
    for p in image_paths:
        name = os.path.splitext(os.path.basename(p))[0]
        loaded = load_gray(p)
        if loaded is None:
            print(f'Warning: failed to load {p}')
            continue
        img, gray = loaded
        kps, des = ORB.detectAndCompute(gray, None)
        if des is None or len(kps) == 0:
            print(f'Warning: no features for {name} ({p})')
            continue
        dataset[name] = {'image': img, 'gray': gray, 'kps': kps, 'des': des}
    print(f'Dataset built: {len(dataset)} objects from {dataset_dir}')
    return dataset

# Ratio test filtering
def good_matches_knn(des1, des2, ratio=0.75):
    if des1 is None or des2 is None:
        return []
    matches = BF.knnMatch(des1, des2, k=2)
    good = []
    for m_n in matches:
        if len(m_n) != 2:
            continue
        m, n = m_n
        if m.distance < ratio * n.distance:
            good.append(m)
    return good

In [ ]:
# Background removal and candidate extraction
def extract_candidates(frame_bgr):
    # Apply background subtraction
    fgmask = backSub.apply(frame_bgr)
    # Clean mask
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, KERNEL_OPEN)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, KERNEL_CLOSE)
    # Threshold to binary
    _, fgmask = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidates = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w * h < MIN_AREA:
            continue
        candidates.append((x, y, w, h))
    return candidates, fgmask

def crop_gray(frame_bgr, bbox):
    x, y, w, h = bbox
    roi = frame_bgr[y:y+h, x:x+w]
    if roi.size == 0:
        return None, None
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    return roi, gray

In [ ]:
# Match candidates to dataset
def identify_candidate(gray_roi, dataset, min_good_matches=12):
    # Compute ORB descriptors for ROI
    kps_roi, des_roi = ORB.detectAndCompute(gray_roi, None)
    if des_roi is None or len(kps_roi) == 0:
        return None, 0
    best_name = None
    best_score = 0
    for name, item in dataset.items():
        good = good_matches_knn(des_roi, item['des'])
        score = len(good)
        if score > best_score:
            best_score = score
            best_name = name
    if best_name is None or best_score < min_good_matches:
        return None, best_score
    return best_name, best_score

In [ ]:
# Main pipeline: build dataset, detect introductions, and track
def run_tracking(video_path=VIDEO_PATH, output_path=OUTPUT_PATH, dataset_dir=DATASET_DIR, min_match=12):
    # Build dataset
    dataset = build_object_dataset(dataset_dir)
    if len(dataset) == 0:
        print('No objects in dataset; aborting.')
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f'Cannot open video: {video_path}')
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS) or 25)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Active trackers per object name
    trackers = {}  # name -> tracker
    bboxes = {}    # name -> last bbox

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1

        # Update existing trackers
        to_remove = []
        for name, tracker in trackers.items():
            ok, bbox = tracker.update(frame)
            if ok:
                bboxes[name] = bbox
                draw_bbox(frame, bbox, (0, 255, 0))
                # Overlay the object name near the bbox
                x, y = int(bbox[0]), int(bbox[1])
                label_y = max(y - 10, 15)
                cv2.putText(frame, f"{name}", (x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 3, cv2.LINE_AA)
                cv2.putText(frame, f"{name}", (x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
            else:
                # mark lost tracker; we'll try to re-detect
                to_remove.append(name)
        for name in to_remove:
            del trackers[name]
            del bboxes[name]

        # If some objects are not tracked, try detecting new introductions
        candidates, _ = extract_candidates(frame)
        for bbox in candidates:
            roi, gray_roi = crop_gray(frame, bbox)
            if gray_roi is None:
                continue
            obj_name, score = identify_candidate(gray_roi, dataset, min_good_matches=min_match)
            if obj_name is None:
                continue
            # If this object is not already tracked, initialize a tracker
            if obj_name not in trackers:
                trk = create_tracker('CSRT')
                ok = trk.init(frame, bbox)
                if ok:
                    trackers[obj_name] = trk
                    bboxes[obj_name] = bbox
                    print(f"[{frame_idx}] '{obj_name}' detected (matches={score}) and tracker initialized at {bbox}")
                    draw_bbox(frame, bbox, (255, 0, 0))
                    # Overlay the object name on initialization
                    x, y = int(bbox[0]), int(bbox[1])
                    label_y = max(y - 10, 15)
                    cv2.putText(frame, f"{obj_name}", (x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 3, cv2.LINE_AA)
                    cv2.putText(frame, f"{obj_name}", (x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
                else:
                    print(f"[{frame_idx}] Failed to init tracker for '{obj_name}'")

        writer.write(frame)

    cap.release()
    writer.release()
    print('Tracking finished. Output saved to:', output_path)

# Example usage (uncomment to run):
# run_tracking()

# Usage

- Place one image per object in `media/photos`. The file name will be used as the object name (e.g., `pawn.png` → `pawn`).
- Set `VIDEO_PATH` in Cell 4 to your game video file.
- Optional: adjust `MIN_AREA` (Cell 4) and `min_match` argument in `run_tracking()` (Cell 8) for your scene.
- Run Cells 4→8 in order. Uncomment the last line in Cell 8 (`run_tracking()`) to execute the pipeline.
- The output video is written to `media/game_track.avi`.